In [ ]:
import torch
import numpy as np
import pandas as pd
from tqdm import tqdm
from torch import nn, optim
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, Dataset

In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())
torch.multiprocessing.freeze_support()

2.7.0+cu118
True


In [ ]:
images = np.load('D:\\ds\\images.npy')  
labels = np.load('D:\\ds\\labels.npy')  
test_images = np.load('D:\\ds\\images_sub.npy')

In [ ]:
BATCH_SIZE = 128
EPOCHS = 30
LR = 1e-3
PATIENCE = 5

In [ ]:
# Аугментации: случайное поворот и смещение, а также изменение яркости и контраста
train_transform = transforms.Compose([
transforms.RandomAffine(degrees=5, translate=(0.05, 0.05)),
transforms.ColorJitter(brightness=0.1, contrast=0.1),
])

# Кастомный датасет с возможностью применения аугментаций
class AugmentedDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

# Нормализация изображений, приведение к тензорам, перестановка каналов
images = torch.tensor(images.astype('float32')/255.0).permute(0,3,1,2)
labels = torch.tensor(labels.astype('int64'))

X_train, X_val, y_train, y_val = train_test_split(
    images, labels, test_size=0.2, stratify=labels.numpy(), random_state=42
)
train_dataset = AugmentedDataset(X_train, y_train, transform=train_transform)
val_dataset = TensorDataset(X_val, y_val)

# Даталоадеры: один с аугментациями, другой без
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0  
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

# Сверточная нейросеть с батч-нормализацией и дропаутом
class DeepCNN(nn.Module):
    def __init__(self, num_classes=26):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(128, 256, 3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.pool(x)
        return self.classifier(x)

# Инициализация модели, загрузка лучших весов
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DeepCNN().to(device)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

# Оптимизатор, функция потерь и scheduler на уменьшение lr при плато
optimizer = optim.Adam(model.parameters(), lr=LR, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)

best_val_acc = 0.0
no_improve = 0

# Цикл обучения: тренировка, валидация, сохранение лучшей модели, early stopping
for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss = 0
    for xb, yb in tqdm(train_loader, desc=f'Epoch {epoch} [Train]'):
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            preds = model(xb).argmax(dim=1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
    
    val_acc = correct / total
    print(f'Epoch {epoch}: Train Loss: {train_loss/len(train_loader):.4f}, Val Acc: {val_acc:.4f}')
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')
        no_improve = 0
    else:
        no_improve += 1
        if no_improve >= PATIENCE:
            break
    
    scheduler.step(val_acc)

# Подготовка тестового датасета и предсказания
test_images = torch.tensor(test_images.astype('float32')/255.0).permute(0,3,1,2)
test_loader = DataLoader(test_images, batch_size=BATCH_SIZE, shuffle=False)


model.load_state_dict(torch.load('best_model.pth'))
model.eval()
all_preds = []
with torch.no_grad():
    for xb in test_loader:
        xb = xb.to(device)
        outputs = model(xb)
        all_preds.extend(outputs.argmax(dim=1).cpu().tolist()) 

submission = pd.DataFrame({
    'id': np.arange(len(all_preds)),
    'Category': all_preds
})
submission.to_csv('submission.csv', index=False)

Epoch 1 [Train]: 100%|██████████| 125/125 [00:21<00:00,  5.94it/s]


Epoch 1: Train Loss: 0.3688, Val Acc: 0.8665


Epoch 2 [Train]: 100%|██████████| 125/125 [00:20<00:00,  6.03it/s]


Epoch 2: Train Loss: 0.3526, Val Acc: 0.8805


Epoch 3 [Train]: 100%|██████████| 125/125 [00:21<00:00,  5.77it/s]


...

Epoch 14: Train Loss: 0.1944, Val Acc: 0.9097


Epoch 15 [Train]: 100%|██████████| 125/125 [00:22<00:00,  5.56it/s]


Epoch 15: Train Loss: 0.1859, Val Acc: 0.9123


Epoch 16 [Train]: 100%|██████████| 125/125 [00:20<00:00,  6.18it/s]


Epoch 16: Train Loss: 0.1870, Val Acc: 0.9130
